In [56]:
import sympy as sp

In [57]:
x = sp.symbols('x')
f01 = x**4/8-x**3/3+2*x/3-1/2
f12 = -x**4/24+x**3/6-x**2/4+x/6-1/24
fn01 = -x**4/8+x**3/6+x**2/4+x/6+1/24
fn12 = x**4/24

f12 = f12.subs(x,x-1)
fn01 = fn01.subs(x,x+1)
fn12 = fn12.subs(x,x+2)

In [58]:
sp.expand(fn12)

x**4/24 + x**3/3 + x**2 + 4*x/3 + 2/3

In [59]:
import sympy
import numpy as np
from scipy.optimize import minimize, LinearConstraint

x = sympy.symbols('x')
f = sympy.Piecewise((f01, x < 1), (f12, x >= 1))

# --- SymPy 阶段 (处理端点约束，建立目标函数) ---

# 定义符号系数
k4, k3, k2 = sympy.symbols('k4, k3, k2') # 3个待优化变量
k1_sym, k0_sym = sympy.symbols('k1, k0') # 被消除的变量

polynomial_base = k4*x**4 + k3*x**3 + k2*x**2 + k1_sym*x + k0_sym
polynomial_prime_base = sympy.diff(polynomial_base, x)

# 1. 定义端点约束方程 P(2)=0 和 P'(2)=0
eq_P2 = sympy.Eq(polynomial_base.subs(x, 2), 0)
eq_Pprime2 = sympy.Eq(polynomial_prime_base.subs(x, 2), 0)

# 2. 从约束中解出 k0 和 k1 的表达式（关于 k4, k3, k2）
constraints_solutions = sympy.solve([eq_P2, eq_Pprime2], (k0_sym, k1_sym))
k0_expr = constraints_solutions[k0_sym]
k1_expr = constraints_solutions[k1_sym]

# 3. 构造只依赖于 k4, k3, k2 的多项式
polynomial_sym = polynomial_base.subs({k0_sym: k0_expr, k1_sym: k1_expr})
polynomial_prime_sym = sympy.diff(polynomial_sym, x) # 新的导数表达式

# 4. 计算总误差 (MSE)
def mse_integrand_sym(polynomial, f):
    return (polynomial - f)**2

error_sym = sympy.integrate(mse_integrand_sym(polynomial_sym, f), (x, 0, 2))

# --- SciPy 阶段 (进行数值优化) ---

# 1. 定义目标函数
k_syms = [k4, k3, k2] # SciPy 优化的变量顺序
mse_func = sympy.lambdify(k_syms, error_sym, 'numpy')

def objective(k):
    # k = [k4, k3, k2]
    return mse_func(*k)

# 2. 定义单调性约束 P'(x) >= 0
N = 500
points = np.linspace(0, 2, N)
monotone_constraints = []

for x_val in points:
    # 约束表达式：P'(x_val) >= 0 (现在 P' 依赖于 k4, k3, k2)
    constraint_expr = polynomial_prime_sym.subs(x, x_val)
    
    # 提取系数，形式为：c4*k4 + c3*k3 + c2*k2
    coeffs = [constraint_expr.coeff(k) for k in k_syms]
    
    # LinearConstraint(A, lb, ub): A.dot(k) >= 0
    monotone_constraints.append(LinearConstraint(
        np.array(coeffs, dtype=float),
        0, # lb (下界)
        np.inf # ub (上界)
    ))

# 3. 运行优化器
# 初始猜测：我们使用之前无约束 k4, k3, k2 的近似值
# 无约束解：k4 ≈ 0.10, k3 ≈ -0.09, k2 ≈ 0.32
initial_guess = np.array([0.1, -0.1, 0.3], dtype=float)

result = minimize(
    objective,
    initial_guess,
    method='SLSQP',
    constraints=monotone_constraints, # 只传入单调性约束
    options={'disp': False, 'maxiter': 1000}
)

# 4. 提取结果
if result.success:
    k_optimized = result.x
    k4_opt, k3_opt, k2_opt = k_optimized
    
    # 计算被消除的 k1 和 k0 的值
    k_subs_dict = {k4: k4_opt, k3: k3_opt, k2: k2_opt}
    k1_opt = k1_expr.subs(k_subs_dict).evalf()
    k0_opt = k0_expr.subs(k_subs_dict).evalf()
    
    # 构造最终多项式 (数值近似)
    polynomial_final = (
        k4_opt * x**4 + k3_opt * x**3 + k2_opt * x**2 + k1_opt * x + k0_opt
    )
    
    print("--- 受 P(2)=0, P'(2)=0 和单调性约束的最优 4 次多项式 (数值近似) ---")
    print(f"P(x) ≈ {polynomial_final}")
    print("\n--- 最优系数 (近似值) ---")
    print(f"k4 ≈ {k4_opt:.16f}")
    print(f"k3 ≈ {k3_opt:.16f}")
    print(f"k2 ≈ {k2_opt:.16f}")
    print(f"k1 ≈ {k1_opt:.16f}")
    print(f"k0 ≈ {k0_opt:.16f}")
    
    # 验证约束
    P2_check = polynomial_final.subs(x, 2).evalf()
    Pprime2_check = sympy.diff(polynomial_final, x).subs(x, 2).evalf()
    
    print(f"\n--- 验证端点约束 ---")
    print(f"P(2) ≈ {P2_check} (应为 0)")
    print(f"P'(2) ≈ {Pprime2_check} (应为 0)")
    print(f"最小均方误差 (MSE) ≈ {result.fun}")
else:
    print(f"优化未成功完成: {result.message}")

--- 受 P(2)=0, P'(2)=0 和单调性约束的最优 4 次多项式 (数值近似) ---
P(x) ≈ -0.0153248731014979*x**4 + 0.157402789607197*x**3 - 0.576410049532422*x**2 + 0.907202662091257*x - 0.522789473286436

--- 最优系数 (近似值) ---
k4 ≈ -0.0153248731014979
k3 ≈ 0.1574027896071969
k2 ≈ -0.5764100495324217
k1 ≈ 0.9072026620912570
k0 ≈ -0.5227894732864360

--- 验证端点约束 ---
P(2) ≈ 0 (应为 0)
P'(2) ≈ 0 (应为 0)
最小均方误差 (MSE) ≈ 7.612548659063911e-05


polyblep很好，但我不满足于只有两个采样的polybelp。
通过一篇论文我得知可以使用B-Spline构造3个、4个采样长的polybelp。然而它包含太多的分支，在不做任何优化下有五个分支，实在是不适合实时规则。通过构造只有正数的多项式B(x)再利用一些max min小技巧以及simd mask技巧可以消除分支，这很好，但我还是不满足。
我查看过monark(reaktor上一个合成器)的corecell，有一个长达八个采样的blep模块，并且由一个7阶多项式直接计算。我开始思考如何也能达到这样的方法。

在振荡器中，我们通过使用phase[0..1]和phase_inc = f/fs来计算振荡器相位。简单的思考，我们知道计算1/phase_inc次就能完成一个周期，那么一个周期的长度在离散时间就是Td = fs/f(采样数)，在连续时间即为Ta = Td/fs = 1/f(秒)，这可以看成我们正在采样一个f频率的连续波形，这也是为什么naive振荡器会出现混叠的原因。
接下来是blep在离散和连续的长度。众所周知，polyblep中有一个计算为phase/dt，其中dt=phase_inc。而phase = dt * n，n=[0..fs/f]，忘掉这些数字，从phase=0开始，只需要计算一次phase就会到达dt从而到达blep的末尾，即那个
```
if (phase_ < dt) {
    ...blep code...(假设-1..0)
}
```
分支的边界
而如果是两个采样的blep，例如B-spline，则naive代码通常为
```
if (phase_ < dt) {
    ...blep code...(假设-1..-0)
}
else if (phase_ < dt * 2) {
    ...blep code...(假设-2..-1)
}
```
从这里我们就能得知blep的离散长度就是它的采样长度，最常见的polyblep是一个长度为2采样的三角形窗，4采样B-spline polyblep(-2..2)是一个长度为4采样的多项式窗。从而得知一半的连续域长度为1/fs秒和2/fs秒。
我们知道在滤波器设计里有许多的窗函数使用，我们可以设计任意采样长的blep使用，只需要使用任意多项式逼近算法(推荐lolremez程序)即可(什么，直接使用原始超越函数吗?)。
现在让我们制作一个4采样的hann窗(使用下面的公式计算，x<-2和x>2均为0)

In [60]:
import sympy as sp
x = sp.Symbol('x')
Impluse = (1+sp.cos(sp.pi*x/2))/4
Step = sp.integrate(Impluse, (x, -2, x))
Impluse

cos(pi*x/2)/4 + 1/4

现在让我们使用任何符号计算软件或者desmos计算这个窗的连续傅里叶变换
我们可以看到在0~0.5之间递减到-inf dB，回想一下blep的连续长度是4/fs(秒)，而计算时的连续长度是4(秒),blep被压缩了1/fs倍，那么傅里叶变换则扩展fs倍。根据这一点，我们就可以知道0~fs/2之间递减，然后继续递减下去，而fs/2之后的频率会全部混叠回0~fs之间，所以我们要尽量减少0.5之后的频率内容。
另外一点，blep也会衰减0~fs/2之间的内容，所以我们应该选择滚降速度快的窗作为基准。
例如如果使用4点的hann就会导致高频被大规模衰减，或者你也可以选择动态改变dt参数，缩放dt为原来的s倍，这会导致blep的截止频率减小s倍从而抑制更多的高频。
https://www.desmos.com/calculator/isemvqu5bv?lang=zh-CN

让我们创建一个6采样长的Blackman–Nuttall，有着100dB的伪影抑制，在奈奎斯特频率处衰减36.69dB
https://www.desmos.com/calculator/mubdmjnyil?lang=zh-CN

In [61]:
import sympy as sp
a0 = 0.3635819
a1 = 0.4891775
a2 = 0.1365995
a3 = 0.0106411
x = sp.Symbol('x')
Impluse = a0 - a1*sp.cos(sp.pi*(x/3+1)) + a2*sp.cos(sp.pi*2*(x/3+1)) - a3*sp.cos(sp.pi*3*(x/3+1))
Normalize = sp.integrate(Impluse, (x,-3,3))
Impluse = Impluse / Normalize
Step = sp.integrate(Impluse, (x,-3,x))
Positive_Blep = Step - 1
sp.expand(Positive_Blep)

0.166666666666667*x + 0.672719819110907*sin(pi*x/3)/pi + 0.0939262240502071*sin(2*pi*x/3)/pi + 0.00487790142101867*sin(pi*x)/pi - 0.5